In [7]:
import math
import torch
import gpytorch
from matplotlib import pyplot as plt
from gpytorch.variational import MeanFieldVariationalDistribution, IndependentMultitaskVariationalStrategy, VariationalStrategy


In [8]:
train_x1 = torch.rand(2000)
train_x2 = torch.rand(2000)

train_y1 = torch.sin(train_x1 * (2 * math.pi)) + torch.randn(train_x1.size()) * 0.2
train_y2 = torch.cos(train_x2 * (2 * math.pi)) + torch.randn(train_x2.size()) * 0.2

In [13]:
jitter = 1e-4
num_latents = 2
learn_inducing_locations = True
num_tasks = 2

class MultitaskGPModel(gpytorch.models.ApproximateGP):
    def __init__(self, x_train, task_indices, y_train, likelihood):
          # MeanField constructs a variational distribution for each output dimension
        variational_distribution = MeanFieldVariationalDistribution(
                                    num_inducing_points=x_train.size(0),
                                    batch_shape=torch.Size([num_latents]),
                                    jitter=jitter
                                )
        
        # LMC constructs MultitaskMultivariateNormal from the base var dist
        variational_strategy = IndependentMultitaskVariationalStrategy(
                            VariationalStrategy(
                                    model=self, 
                                    inducing_points=x_train, 
                                    variational_distribution=variational_distribution, 
                                    learn_inducing_locations=learn_inducing_locations,
                                ),
                            num_tasks=num_tasks,
                            task_dim=-1
                        )
        super().__init__(variational_strategy)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.RBFKernel()

        # We learn an IndexKernel for 2 tasks
        # (so we'll actually learn 2x2=4 tasks with correlations)
        self.task_covar_module = gpytorch.kernels.IndexKernel(num_tasks=2, rank=1)

    def forward(self,x,i):
        mean_x = self.mean_module(x)

        # Get input-input covariance
        covar_x = self.covar_module(x)
        # Get task-task covariance
        covar_i = self.task_covar_module(i)
        # Multiply the two together to get the covariance we want
        covar = covar_x.mul(covar_i)

        return gpytorch.distributions.MultivariateNormal(mean_x, covar)

likelihood = gpytorch.likelihoods.BetaLikelihood()

train_i_task1 = torch.full((train_x1.shape[0],1), dtype=torch.long, fill_value=0)
train_i_task2 = torch.full((train_x2.shape[0],1), dtype=torch.long, fill_value=1)

full_train_x = torch.cat([train_x1, train_x2])
full_train_i = torch.cat([train_i_task1, train_i_task2])
full_train_y = torch.cat([train_y1, train_y2])

# Here we have two iterms that we're passing in as train_inputs
model = MultitaskGPModel(full_train_x, full_train_i, full_train_y, likelihood)

In [15]:
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters
training_iterations = 50
# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.VariationalELBO(likelihood, model, num_data=full_train_y.size(0))

for i in range(training_iterations):
    optimizer.zero_grad()
    output = model(full_train_x, full_train_i)
    loss = -mll(output, full_train_y)
    loss.backward()
    print('Iter %d/50 - Loss: %.3f' % (i + 1, loss.item()))
    optimizer.step()

RuntimeError: Boolean value of Tensor with more than one value is ambiguous